In [1]:
import numpy as np
import growth_simulations as g
import os
import scipy
import time
# import seaborn as sns
import weakref
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats
# sns.set_style("whitegrid")

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [35]:
def jiseon_model_2(l, beta, sigma_l, alpha):
    num = beta*np.log(2-2*alpha+alpha/beta)**2+(1-beta)*(np.log(2-2*alpha+alpha/(1-beta)))**2
    denom = beta*np.log(2-2*alpha+alpha/beta)+(1-beta)*(np.log(2-2*alpha+alpha/(1-beta)))
    return l*(1-sigma_l**2/l**2*(1+0.5*num/denom))

In [25]:
# # Setting simulation parameters
l = np.array([1.0])
td_std = np.array([0.0])  # 1
lambda_std = np.linspace(0.0, 0.3, num=16)  # 11
delta = np.array([1.0])
beta = np.linspace(0.025, 0.5, num=20)  # 20
r = beta/(1-beta)
x=1-2*beta
alpha = np.linspace(0.0,1.0,6)  # 1
epsilon=np.array([0.0])

num_rep = 100  # number of repeats
# # should give 32000 repeats. Run with 800 job array.
# # should take around 8 hours (480min).49899542
par_vals = {'nstep': 900, 'dt': 0.01, 'v_init': 1.0, 'modeltype': 14, 'delta': delta[0], 'lambda': l[0]}
X = [len(td_std), len(lambda_std), len(beta), len(alpha), len(epsilon), num_rep]
i0,i1,i2,i3,i4=0,10,11,0,0
par_vals['td_std']=td_std[i0]
par_vals['lambda_std']=lambda_std[i1]
par_vals['r']=beta[i2]/(1-beta[i2])
par_vals['f']=alpha[i3]
par_vals['epsilon']=epsilon[i4]
par_vals['T2']=0.0
par_vals['exp_n']=4


In [26]:
tic=time.clock()
init_pop = g.starting_popn(par_vals)
par_vals['nstep']=500  # seeding the population with a simulated one
c, obs, [temp_vols, temp_vols_G1] = g.discr_time_1(par_vals, init_pop)
init_pop = g.starting_popn_seeded(c, par_vals)
par_vals['nstep'] = 900  # now we run this simulation for longer with a better seeded population.
c, obs, [temp_vols, temp_vols_G1] = g.discr_time_1(par_vals, init_pop)
print 'Time taken ={0}s'.format(np.around(time.clock()-tic,2))

Time taken =118.88s


In [27]:
print par_vals
temp = scipy.stats.linregress(obs[1][400:], np.log(obs[7][400:]))
print temp[0]  # GR vol
print jiseon_model_2(par_vals['lambda'], par_vals['r']/(1+par_vals['r']), par_vals['lambda_std'], 1-par_vals['f']/2)

{'td_std': 0.0, 'lambda_std': 0.2, 'f': 0.0, 'epsilon': 0.0, 'exp_n': 4, 'T2': 0.0, 'v_init': 1.0, 'modeltype': 14, 'r': 0.4285714285714286, 'delta': 1.0, 'dt': 0.01, 'nstep': 900, 'lambda': 1.0}
0.9778763935526367
0.9771533043672094


In [28]:
i6=1
xv = [obj.vb for obj in c if obj.celltype == i6 and obj.tb>400* par_vals['dt'] * np.log(2)/par_vals['lambda']]        
yv = [obj.vd for obj in c if obj.celltype == i6 and obj.tb>400* par_vals['dt'] * np.log(2)/par_vals['lambda']]
temp = scipy.stats.linregress(xv, yv)
print temp[0]

0.08886322701054931
